In [1]:
import jieba
import os
import re

#set jieba
f1 = open('/Users/NRB/Documents/lawsnote/build_dictionary/mydict_final.txt',encoding='UTF-8')
jieba.set_dictionary("/Users/NRB/Documents/lawsnote/dict.txt.big")
jieba.load_userdict(f1)
f1.close()

Building prefix dict from /Users/NRB/Documents/lawsnote/dict.txt.big ...
Loading model from cache /var/folders/qp/9h4qdt_d2_77rprb8x3y43y80000gn/T/jieba.u3e74f9a505573d512c78bba50d9005d6.cache
Loading model cost 1.654 seconds.
Prefix dict has been built succesfully.


以智慧財產法院的判決為例，首先先將檔案逐一讀取，擷取需要的部分(judegement)，然後去除空白和附表，接著再用結巴斷詞，最後存成一個txt檔，之後訓練model用。
判決download於lawsnote開放法學資料庫，格式為json檔，有7492篇。

In [6]:
#read files
dirPath = r"/Users/NRB/Documents/lawsnote/智慧財產法院"
result = [f for f in os.listdir(dirPath) if os.path.isfile(os.path.join(dirPath, f))]
os.chdir(dirPath)

#write a file with cutted word
output = open('/Users/NRB/Documents/lawsnote/judgement_seg_ip.txt', 'w', encoding='UTF-8')
counter=0
present=0
for i in range(len(result)):
    words = []
    f = open(result[i], 'r', newline="", encoding='UTF-8',errors='replace')
    text = f.read()
    f.close()

    q = r"judgement[\S].*type"
    raw = re.findall(q, text)
    remov = r"\r\n|\\n|\\r|\s+|\s|"
    

    try:
        k = re.sub(remov, "", raw[0])
    except:
        print(raw,result[i])
        k=0
    k = re.sub(r"judgement\":\" ","", k)
    
    #remove appendixes bcoz it is meaningless 
    if re.search(r"附表\S{,5}：\S{,15}┌─.*\"type",k):
        k = re.sub(r"附表\S{,5}：\S{,15}┌─.*\"type","",k)
    else:
        k = re.sub(r"type","",k)

    words = jieba.lcut(k, cut_all=False)
    for word in words:
        output.write(word+' ')
    output.write('\n')

    #print some articles as examples 
    counter += 1
    if counter % int(len(result)/10) == 0:
        print("已完成",present,"%")
        present+=10
        print(words)

output.close()



已完成 0 %
['judgement', '"', ':', '"', '智慧財產法院', '民事判決', '98', '年度', '民專訴', '字', '第', '156', '號', '原告', '甲', '○', '○', '訴訟代理人', '盧俊', '誠', '律師', '複代理人', '顏文正', '律師', '輔佐人', '乙', '○', '○', '（', '兼', '送達代收', '被告', '臺灣', '山葉', '機車', '工業', '股份有限公司', '兼上', '一人', '法定代理人', '丙', '○', '○', '共同', '訴訟代理人', '李文傑', '律師', '郭家君', '律師', '上列', '當事人', '間', '侵害專利權', '有關', '財產權', '爭議', '等', '事件', '，', '本院', '於', '99', '年', '3', '月', '9', '日', '言詞辯論終結', '，', '判決', '如下', '：', '主文', '原告', '之訴', '及', '假執行', '之', '聲請', '均', '駁回', '。', '訴訟費用', '由', '原告', '負擔', '。', '事實', '及', '理由', '一', '、', '原告', '起訴', '主張', '其', '於', '民國', '95', '年', '8', '月', '10', '日以', '自行', '研創', '之', '「', '車輛', '儀表', '之', '液晶面板', '」', '向', '經濟部', '智慧財產局', '（', '下稱', '智慧', '局', '）', '提出', '新型專利', '之', '申請', '，', '業已', '取得', '新型', '第', 'M306192', '號', '專利權', '（', '下稱', '系爭', '專利', '）', '，', '專利', '期間', '自', '96', '年', '2', '月', '11', '日至', '105', '年', '8', '月', '9', '日止', '，', '並', '取得', '技術', '報告', '，', '報告', '結果', '系爭', '專利', '係', '一', '合法

KeyboardInterrupt: 

印幾個出來看看效果如何